In [1]:
import requests
import pandas as pd
import numpy as np
import urllib.request

In [2]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r *1000

In [3]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])
traffic_image_df['Count']=np.random.uniform(low=0, high=20, size=(87,)).astype(int)
traffic_image_df['is_jam']=0

In [4]:
traffic_image_df=traffic_image_df.merge(pd.read_csv('traffic_camera_region_roadname.csv',converters={'CameraID':str}),'left','CameraID')

In [5]:
traffic_image_df

,CameraID,Latitude,Longitude,ImageLink,Count,is_jam,Region,roadname
0,1001,1.295313,103.871146,https://dm-traffic-camera-itsc.s3.ap-southeast...,1,0,Central,East Coast Parkway (ECP)
1,1002,1.319541,103.878563,https://dm-traffic-camera-itsc.s3.ap-southeast...,9,0,Central,Pan-Island Expressway (PIE)
2,1003,1.323957,103.872858,https://dm-traffic-camera-itsc.s3.ap-southeast...,12,0,Central,Pan-Island Expressway (PIE)
3,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,13,0,Central,Kallang Way Flyover
4,1005,1.363520,103.905394,https://dm-traffic-camera-itsc.s3.ap-southeast...,2,0,East,Kallang-Paya Lebar Expressway (KPE)
...,...,...,...,...,...,...,...,...
82,9702,1.394741,103.817971,https://dm-traffic-camera-itsc.s3.ap-southeast...,7,0,North,Seletar Expressway (SLE)
83,9703,1.422857,103.773005,https://dm-traffic-camera-itsc.s3.ap-southeast...,9,0,North,Seletar Expressway (SLE)
84,9704,1.422143,103.795421,https://dm-traffic-camera-itsc.s3.ap-southeast...,12,0,North,Seletar Expressway (SLE)
85,9705,1.426277,103.787166,https://dm-traffic-camera-itsc.s3.ap-southeast...,8,0,North,Seletar Expressway (SLE)


In [6]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])


# NEA API
For some reason this one need to constantly call and keep track of previous results. Sometimes calls does not return all data.
Some names are given as S123, etc, looking to try OneMap reverse geocoding to get road name from lat long.

In [7]:
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')
weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

weather_df['latitude']=weather_df['location'].apply(lambda x: x['latitude'])
weather_df['longitude']=weather_df['location'].apply(lambda x: x['longitude'])
weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
weather_df['timestamp']=pd.to_datetime(weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall'})

weather_df=weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
weather_df=weather_df.drop(['id','device_id','station_id','location'],axis=1)

# Calculations


In [8]:
traffic_image_df['key']=0
traffic_incidents_df['key']=0
weather_df['key']=0

In [9]:
nearest_incidents=traffic_image_df.merge(traffic_incidents_df,'outer','key')
nearest_incidents['incident_distance_from_id']=(np.vectorize(haversine)(nearest_incidents['Latitude_x'],nearest_incidents['Longitude_x'],nearest_incidents['Latitude_y'],nearest_incidents['Longitude_y']))
nearest_incidents=nearest_incidents[nearest_incidents['incident_distance_from_id']<500].sort_values('incident_distance_from_id')
nearest_incidents=nearest_incidents[['CameraID','Message']]

In [10]:
final_df=traffic_image_df.merge(weather_df,'outer','key')

final_df['distance_from_id']=(np.vectorize(haversine)(final_df['Latitude'],final_df['Longitude'],final_df['latitude'],final_df['longitude']))
final_df=final_df.sort_values('distance_from_id').groupby('CameraID').head(1)[['CameraID','Latitude','Longitude','Region','rainfall','ImageLink','roadname','Count','is_jam']]
final_df=final_df.sort_values('CameraID').reset_index(drop=True)


In [11]:
final_df.to_csv('main_df.csv',index=False)
nearest_incidents.to_csv('traffic_incidents.csv',index=False)

# END